In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv("./gurgaon_properties_cleaned_v1.csv").drop_duplicates()
data.head()

# Price

In [ ]:
data["price"].describe()

In [ ]:
sns.distplot(data["price"])
plt.show()

In [ ]:
# price is not normally distributed so, plotting a boxplot
sns.boxplot(data["price"])
plt.show()

In [ ]:
# using IQR method to detect outliers
Q1 = data["price"].quantile(0.25)
Q3 = data["price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - (1.5*IQR)
upper_bound = Q3 + (1.5*IQR)

filt = (data["price"] < lower_bound) | (data["price"] > upper_bound)
price_outlier_df = data[filt]

num_outliers = price_outlier_df.shape[0]
num_outliers, price_outlier_df["price"].describe()

- Based on the price column -> there are some genuine outliers, i.e., there are some actual properties that are worth the high price but there are some data errors as well

# price per square feet

In [ ]:
data["price_per_sqft"].describe()

In [ ]:
sns.distplot(data["price_per_sqft"])
plt.show()

In [ ]:
sns.boxplot(data["price_per_sqft"])
plt.title("price_per_sqft")
plt.show()

In [ ]:
# using IQR method to detect outliers
Q1 = data["price_per_sqft"].quantile(0.25)
Q3 = data["price_per_sqft"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - (1.5*IQR)
upper_bound = Q3 + (1.5*IQR)

filt = (data["price_per_sqft"] < lower_bound) | (data["price_per_sqft"] > upper_bound)
pps_outlier_df = data[filt]

num_outliers = pps_outlier_df.shape[0]
num_outliers, pps_outlier_df["price_per_sqft"].describe()

In [ ]:
pps_outlier_df.head(10)

- Turns out that properties where area < 1000, the area is in sq yards and since , price per sqft has been calculated based on area , price per sqft is inaccurate.

In [ ]:
pps_outlier_df[pps_outlier_df['area'] < 1000]

In [ ]:
pps_outlier_df["area"] = pps_outlier_df["area"].apply(lambda x: x * 9 if x<1000 else x)
pps_outlier_df["builtup_area"] = pps_outlier_df["builtup_area"].apply(lambda x: x * 9 if x<1000 else x)
pps_outlier_df["carpet_area"] = pps_outlier_df["carpet_area"].apply(lambda x: x * 9 if x<1000 else x)

In [ ]:
data.update(pps_outlier_df)

In [ ]:
data["price_per_sqft"] = (data["price"] * 10000000) / data["area"]

In [ ]:
sns.boxplot(data["price_per_sqft"])
plt.title("price per sqft after converting sq yards to sqft")
plt.show()

In [ ]:
data[data["price_per_sqft"] > 50000].shape[0]

- There are 5 properties where price per sqft is > 50000, so dropping them

In [ ]:
data = data[data["price_per_sqft"] <= 50000]

In [ ]:
sns.boxplot(data["price_per_sqft"])
plt.title("price per sqft after outlier removal")
plt.show()

# Area 

In [ ]:
sns.distplot(data["area"])
plt.show()

In [ ]:
data[data["area"] > 100000]

- There are properties where area is >100000, but it is not justified by the price which ranges from 31 lakhs to 5 crores.
- so, these are data errors, hence dropping them

In [ ]:
data = data[data["area"] < 100000]

In [ ]:
# analyzing data where area is >10000

data[data["area"] > 10000]

- These are rows where the area is > 10000 but the price is in lakhs, which is illogical , so dropping those -> 452, 1559, 1562, 1563, 1564, 2469

In [ ]:
data.drop(index=[452, 1559, 1562, 1563, 1564, 2469], inplace=True)

In [ ]:
data[data["area"] > 10000]

- in row `63`, house with area 22498 sqft is costing just 6.25 crore. but on 99acres, properties with area around 6000 sqft itself cost more than 6 crores. so this is a data error.
- in row `1860`, flat in MRM Golfestate has area 11000 sqft and costs 13.20 Cr. but a flat there with area around 5207 sqft costs 5.2 Cr. So, even by extension this seems to be  a data error.
- and so is with rows `2555` and `2883`
- so , dropping them

In [ ]:
data.drop(index=[63,1860,2555,2883], inplace=True)

In [ ]:
sns.distplot(data["area"])
plt.title("Area after outlier removal")
plt.show()

# bedRoom

In [ ]:
data["bedRoom"].describe()

In [ ]:
sns.distplot(data["bedRoom"])
plt.show()

In [ ]:
data[data["bedRoom"]>10]

- In rows where number of bedrooms is > 10, it doesn't seem logical that the specified area could have those many bedrooms. 
- and we're gonna keep bedrooms as a part of our predictor model, so we are dropping these rows

In [ ]:
data = data[data["bedRoom"] <= 10]

In [ ]:
sns.distplot(data["bedRoom"])
plt.title("Distribution of bedRoom after outlier removal")
plt.show()

# bathrooms

In [ ]:
sns.distplot(data["bathroom"])
plt.show()

In [ ]:
data[data["bathroom"] > 10]

- In rows where number of bathrooms > 10 (`158`, `182`), the number of bathrooms makes sense given the area and price of the properties. so, keeping those

# super built up area

In [ ]:
sns.distplot(data["super_builtup_area"])
plt.show()

In [ ]:
sns.boxplot(data["super_builtup_area"])
plt.show()

# Builtup area

In [ ]:
sns.distplot(data["builtup_area"])
plt.show()

In [ ]:
sns.boxplot(data["builtup_area"])
plt.show()

# Carpet area

In [ ]:
sns.distplot(data["carpet_area"])
plt.show()

In [ ]:
sns.boxplot(data["carpet_area"])
plt.show()

In [ ]:
data[data["carpet_area"] > 6000]

# area to bedroom ratio
There are certain properties where the area is very small say, around 1200 sq ft but the number of bedrooms is quite high like 9 or 10

In [ ]:
data= pd.read_csv("./gurgaon_properties_post_outlier_removal.csv").drop(columns="Unnamed: 0")
data.head()

In [ ]:
(data["area"] / data["bedRoom"]).quantile(0.05)

if area to bedroom ratio is less than 295, it can be an outlier

In [ ]:
data["area_to_bedroom_ratio"] = data["area"] / data["bedRoom"]

In [ ]:
ab_outlier_df = data[data["area_to_bedroom_ratio"] < 295]

In [ ]:
ab_outlier_df.to_csv("ab_outlier.csv")

- Dropping rows where area to bedroom ratio < 100

In [ ]:
data = data[data["area_to_bedroom_ratio"] > 100]

- There are properties with multiple floors. so, say there are 9 bedrooms but 3 floors, then it is possible to have 3 bedrooms on each floor and the plot area might actually justify it. so, to get the number of bedrooms per floor, divide bedRooms by floorNum

In [ ]:
data["bedRoom"] =round(data["bedRoom"] / data["floorNum"])

In [ ]:
# recalculating the area to bedroom ratio
data["area_to_bedroom_ratio"] = data["area"] / data["bedRoom"]

In [ ]:
data[data["area_to_bedroom_ratio"] < 295]

- row 71 -> 9 bedrooms in 2471 sqft -> illogical
- rows = `101`, `102`, `1470`, `2181`, `2351` -> there's 3 or 4 bedrooms in areas <= 800 ft on a single floor. -> dropping them

In [ ]:
data.drop(index=[71,101,102,1470,2181,2351], inplace=True)

In [ ]:
data[data["area_to_bedroom_ratio"] < 295]

- in rows `115`, `165`, `520` -> there's again a large number of bedrooms in small area and a single floor -> dropping them

In [ ]:
data.drop(index=[115,165,520], inplace=True)

In [ ]:
sns.lmplot(data=data, x="area", y="bedRoom")
plt.show()

# exporting as csv

In [ ]:
data.shape

In [ ]:
data.to_csv("gurgaon_properties_after_outlier_treatment.csv")